In [1]:
from src.data.email_analyzer import EmailAnalyzer
import pandas as pd
import tqdm

In [2]:
### Test des fonctions:

mbox_single_file_path = 'data/processed/mailbox_cecile/AG.mbox'
mbox_path = 'data/processed/mailbox_cecile/'
TEST_SAMPLE_PATH = 'data/processed/real_test_sample/'
db_path = 'data/Projects/database.duckdb'

In [3]:
analyzer =EmailAnalyzer(db_path)
analyzer.get_email_summary()

df_db_cleaned = analyzer.export_to_dataframe()
df_db_cleaned

,message_id,subject,from,to,date,folder,attachment_count,body
0,<0806fb4275064af9ab8141693efd9205@archivistes....,RE: Facture MIC - facilitation Groupe de trava...,"AAF - Anne Clerc, déléguée générale",Myriam FAVREAU,2020-12-01 10:46:05,Archive,6,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
1,<c489e19e47a740ff9d5b9fcd3afb2f96@nouvelle-aqu...,RE: Facture MIC - facilitation Groupe de trava...,Myriam FAVREAU,"AAF - Anne Clerc, déléguée générale",2020-12-01 09:53:54,Archive,5,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
2,<002b01d6c7c4$5e9fb290$1bdf17b0$@coomic.coop>,RE: Facture MIC - facilitation Groupe de trava...,Vincent Taillandier - Les MIC,"AAF - Anne Clerc, déléguée générale, Myriam FA...",2020-12-01 09:28:45,Archive,3,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
3,<014d01d6f97f$b6450df0$22cf29d0$@archivistes.q...,Table-ronde Congrès AAQ 2021 : Archives de la ...,Louis Germain,"mldubois@valorescence.be, coordination@archivi...",2021-02-02 16:23:16,Archive,2,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
4,<ecd898b3f7f04115b785e02a95a77bfb@archivistes....,RE: Facture MIC - facilitation Groupe de trava...,"AAF - Anne Clerc, déléguée générale","Vincent Taillandier - Les MIC, Myriam FAVREAU,...",2020-12-01 07:51:55,Archive,3,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
5,<003a01d6c740$6a12f530$3e38df90$@coomic.coop>,Facture MIC - facilitation Groupe de travail R...,Vincent Taillandier - Les MIC,"AAF - Anne Clerc, déléguée générale, Myriam FA...",2020-11-30 17:44:10,Archive,4,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
6,<8aa028c40ef7146ed18990fc0c9e7a19@archivistes.be>,Re: Table-ronde Congrès AAQ 2021 : Archives de...,coordination@archivistes.be,Louis Germain,2021-02-12 11:33:13,Archive,4,"<html><head>\n<meta http-equiv=""Content-Type"" ..."
7,<01ea01d700bc$2453c3c0$6cfb4b40$@archivistes.q...,RE: Table-ronde Congrès AAQ 2021 : Archives de...,Louis Germain,AAF - Céline Guyon,2021-02-11 21:23:29,Archive,1,"<meta http-equiv=""Content-Type"" content=""text/..."
8,<CE817F92-BE3D-42A0-A833-A08C3B1E7C4A@free.fr>,Communiqué,Agnès Vatican,AAF - Céline Guyon,2020-10-18 10:18:49,Archive,0,"Bonjour Céline, l’AAF a-t-elle prévu de faire ..."
9,<cf77b68bfb2844369ec522a5aac4205a@archivistes....,RE: Facture MIC - facilitation Groupe de trava...,"AAF - Anne Clerc, déléguée générale","Vincent Taillandier - Les MIC, Myriam FAVREAU",2020-12-01 12:23:54,Archive,4,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."


In [16]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v2-moe")
model = AutoModel.from_pretrained("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# Prepare your sentences
sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

# Tokenize the input
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Get the model output
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize the embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Print the embeddings
print(embeddings)


KeyboardInterrupt: 

In [ ]:
from nomic import embed
import torch
import torch.nn.functional as F

# Your sentences
sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

# Generate embeddings using the correct function
response = embed.text(
    texts=sentences,
    model='nomic-embed-text-v1.5',  # Adjust the model name if needed
    task_type='search_document'
)

# Extract embeddings from the response
embeddings_np = response['embeddings']

# Convert to PyTorch tensor
embeddings = torch.tensor(embeddings_np)

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Print shape
print(embeddings.shape)

# Cosine similarity between the two
similarity = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(similarity)



ValueError: You have not configured your Nomic API token. Run `nomic login` to configure.

In [ ]:
from nomic import embed
import torch
import torch.nn.functional as F

# Your sentences
sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

# Get embeddings via Nomic
# This returns a numpy array
embeddings_np = embed(text=sentences, model='nomic-embed-text-v2.0')

# Convert to PyTorch tensor
embeddings = torch.tensor(embeddings_np)

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Print shape
print(embeddings.shape)
# Should print torch.Size([2, 768]) or [2, 2048] depending on model

# Cosine similarity between the two
similarity = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(similarity)




TypeError: 'module' object is not callable

In [6]:
from sentence_transformers import SentenceTransformer
import torch
# Add this near the top of your script
# torch.jit.set_fusion_strategy([("STATIC", 0)])

device = torch.device('cpu')

model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
sentences = ["Hello!", "¡Hola!"]
embeddings = model.encode(sentences, prompt_name="passage")
print(embeddings.shape)
# (2, 768)

similarity = model.similarity(embeddings[0], embeddings[1])
print(similarity)
# tensor([[0.9118]])


/opt/conda/lib/python3.12/site-packages/megablocks/layers/weight_parallel.py:66: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/opt/conda/lib/python3.12/site-packages/megablocks/layers/weight_parallel.py:92: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/opt/conda/lib/python3.12/site-packages/megablocks/layers/weight_parallel.py:134: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/opt/conda/lib/python3.12/site-packages/megablocks/layers/weight_parallel.py:180: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/o

IndexError: map::at

In [ ]:
import os
# Set these before importing any ML libraries
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Then import your libraries
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True, device="cpu")
model = SentenceTransformer("all-MiniLM-L6-v2")  # A simpler model for testing
sentences = ["Hello!", "¡Hola!"]
embeddings = model.encode(sentences, prompt_name="passage")

RuntimeError: Expected x.is_cuda() to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [7]:
import os
# Set these before importing any ML libraries
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Then import your libraries
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True, device="cpu")

sentences = ["Hello!", "¡Hola!"]
embeddings = model.encode(sentences, prompt_name="passage")

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v2-moe")
model = AutoModel.from_pretrained("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
model.eval()
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings.shape)
# torch.Size([2, 768])

similarity = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(similarity)
# tensor(0.9118)


IndexError: map::at

In [ ]:
from nomic import embed

sentences = ['search_document: Hello!', 'search_document: ¡Hola!']
embeddings = embed(text=sentences, model='nomic-embed-text-v2.0')

In [11]:
# Add this near the top of your script


import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

torch.cuda.set_per_process_memory_fraction(0.8)  # Use only 80% of available GPU memory

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v2-moe")

# Load model with trust_remote_code and force fresh download
model = AutoModel.from_pretrained(
    "nomic-ai/nomic-embed-text-v2-moe",
    trust_remote_code=True,
    force_download=True
)

# Move model to the appropriate device
model = model.to(device)

sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

encoded_input = tokenizer(
    sentences,
    padding='max_length',        # Force full padding
    truncation=True,
    max_length=10,               # Set a max length (adjust if needed)
    return_tensors='pt'
)

# Move input tensors to the same device as the model
encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

model.eval()

print("encoded input:", encoded_input)
# print("input_ids:", input_ids)


with torch.no_grad():
    model_output = model(**encoded_input, matryoshka_dim=768)
    # model_output = model(**encoded_input)
    

embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings.shape)
# torch.Size([2, 768])

similarity = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(similarity)
# tensor(0.9118)

encoded input: {'input_ids': tensor([[     0,  33938,    454, 117432,     12,  35378,     38,      2,      1,
              1],
        [     0,  33938,    454, 117432,     12,  14701, 113251,     11,     38,
              2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


IndexError: map::at

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v2-moe")
model = AutoModel.from_pretrained("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

sentences = ['search_document: Hello!', 'search_document: ¡Hola!']

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
model.eval()
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings.shape)
# torch.Size([2, 768])

similarity = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(similarity)
# tensor(0.9118)


In [ ]:
# weird suggest:

# tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-bert-2048")
# model = AutoModel.from_pretrained("nomic-ai/nomic-bert-2048", trust_remote_code=True)

SyntaxError: invalid syntax (3408181778.py, line 1)